Import the data from Edge Impulse. You can obtain the URL from the Dashboard, right-click on the download icon next to 'Spectral features data' and 'Spectral features labels', and click **Copy link location**.

In [1]:
import numpy as np
import requests

API_KEY = 'ei_9eedce842a674656748bf65a19f0e2a80cc867cde21a7810354f75a4fb565a3d'

X = (requests.get('https://studio.edgeimpulse.com/v1/api/24007/training/9/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/24007/training/9/y', headers={'x-api-key': API_KEY})).content
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)

Store the data in a temporary file, and load it back through Numpy.

In [1]:
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

Define our labels and split the data up in a test and training set:

In [2]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "noise", "unknown", "yes" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def set_batch_size(batch_size, train_dataset, validation_dataset):
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return train_dataset, validation_dataset

callbacks = []


Train the model:

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

# model architecture
model = Sequential()
model.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(classes, activation='softmax', name='y_pred'))

# this controls the learning rate
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset, validation_dataset = set_batch_size(BATCH_SIZE, train_dataset, validation_dataset)

# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_dataset, epochs=100, validation_data=validation_dataset, verbose=2, callbacks=callbacks)


Epoch 1/100
90/90 - 2s - loss: 1.0203 - accuracy: 0.5149 - val_loss: 0.7224 - val_accuracy: 0.7159
Epoch 2/100
90/90 - 0s - loss: 0.6339 - accuracy: 0.7483 - val_loss: 0.4231 - val_accuracy: 0.8324
Epoch 3/100
90/90 - 0s - loss: 0.4441 - accuracy: 0.8250 - val_loss: 0.3356 - val_accuracy: 0.8807
Epoch 4/100
90/90 - 0s - loss: 0.3689 - accuracy: 0.8556 - val_loss: 0.2891 - val_accuracy: 0.9034
Epoch 5/100
90/90 - 0s - loss: 0.3380 - accuracy: 0.8715 - val_loss: 0.2696 - val_accuracy: 0.9148
Epoch 6/100
90/90 - 0s - loss: 0.3283 - accuracy: 0.8823 - val_loss: 0.2486 - val_accuracy: 0.9219
Epoch 7/100
90/90 - 0s - loss: 0.2977 - accuracy: 0.8854 - val_loss: 0.2334 - val_accuracy: 0.9304
Epoch 8/100
90/90 - 0s - loss: 0.2862 - accuracy: 0.8958 - val_loss: 0.2255 - val_accuracy: 0.9318
Epoch 9/100
90/90 - 0s - loss: 0.2612 - accuracy: 0.9056 - val_loss: 0.2119 - val_accuracy: 0.9318
Epoch 10/100
90/90 - 0s - loss: 0.2494 - accuracy: 0.9073 - val_loss: 0.2136 - val_accuracy: 0.9318
Epoch 11/

90/90 - 0s - loss: 0.1725 - accuracy: 0.9344 - val_loss: 0.2088 - val_accuracy: 0.9304
Epoch 84/100
90/90 - 0s - loss: 0.1719 - accuracy: 0.9340 - val_loss: 0.2241 - val_accuracy: 0.9276
Epoch 85/100
90/90 - 0s - loss: 0.1680 - accuracy: 0.9361 - val_loss: 0.2010 - val_accuracy: 0.9389
Epoch 86/100
90/90 - 0s - loss: 0.1753 - accuracy: 0.9358 - val_loss: 0.2166 - val_accuracy: 0.9261
Epoch 87/100
90/90 - 0s - loss: 0.1855 - accuracy: 0.9309 - val_loss: 0.2335 - val_accuracy: 0.9148
Epoch 88/100
90/90 - 0s - loss: 0.1756 - accuracy: 0.9351 - val_loss: 0.2082 - val_accuracy: 0.9261
Epoch 89/100
90/90 - 0s - loss: 0.1725 - accuracy: 0.9406 - val_loss: 0.2135 - val_accuracy: 0.9205
Epoch 90/100
90/90 - 0s - loss: 0.1872 - accuracy: 0.9330 - val_loss: 0.2200 - val_accuracy: 0.9233
Epoch 91/100
90/90 - 0s - loss: 0.1822 - accuracy: 0.9365 - val_loss: 0.2043 - val_accuracy: 0.9290
Epoch 92/100
90/90 - 0s - loss: 0.1793 - accuracy: 0.9354 - val_loss: 0.2158 - val_accuracy: 0.9247
Epoch 93/100


In [13]:
# Save the model to disk
model.save('saved_model')
weights_list = model.get_weights()
np.savez('weights_float32.npz', *weights_list)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 50, 13)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 50, 8)             320       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 25, 8)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 8)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 25, 16)            400       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 16)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 16)            0

In [25]:
weights_archive = np.load('weights_float32.npz')
weights_list = [weights_archive['arr_{}'.format(i)] for i in range(len(weights_archive))]
#weights_list = model.get_weights()
for weights in weights_list:
    print(weights.shape)
conv1_weights = weights_list[0]
conv1_biases = weights_list[1]
conv2_weights = weights_list[2]
conv2_biases = weights_list[3]
dense_weights = weights_list[4]
dense_biases = weights_list[5]

(3, 13, 8)
(8,)
(3, 8, 16)
(16,)
(208, 3)
(3,)


In [7]:
print(X.shape)

(3613, 650)


In [8]:
from scipy.special import softmax

def relu(x):
    return np.maximum(x, 0)

def conv1d_single_kernel(x, weights, bias):
    '''Perform convolution of a input feature map with a single filter and bias.
    
    featuremap dims are (time, n_coeffs), so (50, 13)
    '''
    out = np.zeros(x.shape[0], dtype=np.float32)  # output length is the same
    x_pad = np.zeros((x.shape[0]+2, x.shape[1]), dtype=np.float32)
    x_pad[1:-1,:] = x  # zero padding
    for i in range(x.shape[0]):
        section = x_pad[i:i+3,:]
        out[i] = relu(np.sum(section * weights) + bias)
    return out

def conv1d_multi_kernel(x, weights, biases):
    n_kernels = weights.shape[2]
    out = np.zeros((x.shape[0], n_kernels), dtype=np.float32)
    for i in range(n_kernels):
        kernel = weights[:,:,i]
        out[:,i] = conv1d_single_kernel(x, kernel, biases[i])
    return out

def max_pool_1d(x):
    out = np.zeros((int(np.ceil(x.shape[0] / 2)), x.shape[1]), dtype=np.float32)
    for j in range(x.shape[1]):
        for i in range(int(np.floor(x.shape[0] / 2))):
            out[i, j] = np.maximum(x[2*i, j], x[2*i+1, j])
        if (x.shape[0] % 2 == 1):
            out[-1, j] = x[-1, j]
    return out

def dense(x, weights, biases):
    x = x.flatten()
    out = softmax(np.matmul(x, weights) + biases)
    return out

def classify(features):
    out = np.zeros((features.shape[0], 3), dtype=np.float32)
    for i in range(features.shape[0]):
        x = features[i]
        x = x.reshape((int(x.size / 13), 13))
        conv1 = conv1d_multi_kernel(x, conv1_weights, conv1_biases)
        max_pool1 = max_pool_1d(conv1)
        conv2 = conv1d_multi_kernel(max_pool1, conv2_weights, conv2_biases)
        max_pool2 = max_pool_1d(conv2)
        dense1 = dense(max_pool2, dense_weights, dense_biases)
        out[i] = dense1
    return out

In [9]:
my_pred = classify(X[:20,:])
my_pred.shape

In [10]:
#model.predict(X[:4,:])

model2 = Sequential()
model2.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model2.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
model2.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model2.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model2.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model2.add(Flatten())
model2.add(Dense(classes, activation='softmax', name='y_pred'))
model2.set_weights(weights_list)
#model2.summary()
keras_pred = model2.predict(X[:20,:])

In [11]:
diff = my_pred - keras_pred
diff

array([[ 7.4505806e-09, -5.9604645e-08,  1.8626451e-09],
       [ 9.3132257e-10, -5.9604645e-08, -4.4703484e-08],
       [-2.4214387e-08,  1.1920929e-07, -1.8917490e-10],
       [ 2.6193447e-10, -1.1920929e-07, -5.8207661e-10],
       [ 5.9604645e-08, -1.0430813e-07, -1.8626451e-09],
       [-6.4028427e-10,  0.0000000e+00,  6.5192580e-09],
       [ 1.1641532e-10,  0.0000000e+00,  3.0559022e-10],
       [ 6.9849193e-10, -1.1920929e-07,  7.4505806e-09],
       [ 1.1937118e-12,  0.0000000e+00,  6.1845640e-11],
       [ 9.3132257e-09, -5.9604645e-08,  4.4703484e-08],
       [ 5.1222742e-09,  5.9604645e-08,  5.1222742e-09],
       [ 1.3969839e-09, -5.9604645e-08,  4.2632564e-13],
       [-4.3200998e-12,  5.9604645e-08,  0.0000000e+00],
       [-2.2351742e-08,  5.9604645e-08,  2.9802322e-08],
       [-1.4901161e-08,  5.9604645e-08, -1.4901161e-07],
       [-4.4703484e-08,  0.0000000e+00, -1.5133992e-09],
       [-1.4901161e-08,  0.0000000e+00, -4.4703484e-08],
       [ 1.6734703e-10,  0.0000

In [169]:
import matplotlib.pyplot as plt
plt.imshow(diff)